In [ ]:
!pip install bs4

#__IMPORTAÇÃO DAS BIBLIOTECAS__


In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd 

#__REQUISIÇÃO E SELEÇÃO__

In [ ]:
## Função que realiza o web scrap

## Criada função que recebe como parâmetro um estado tipo string, retornando um dicionário python

def consulta_info_estado(estado: str) -> dict:
    
    ## Exibindo na tela os estados consultados
    print(f"Consultando informações do {estado}...")
    
    ## Salvando nossa url que recebe uma interpolação da sigla do estado a ser consultado
    url_estado = f"https://www.ibge.gov.br/cidades-e-estados/{estado}.html"
    
    ## Com o método get() do requests recebemos a url de cada estado
    paginas = requests.get(url_estado)
    
    
    ## Acessada cada pagina nós transformamos seu conteúdo em um objeto BeautifulSoup e aplicamos um html.parser
    soup = BeautifulSoup(paginas.content, "html.parser")
    
    ## No site do IBGE eu analisei no código HTML que os dados que eu quero são contidos na classe indicador
    ## Então eu utilizo o método select() do BeautifulSoup para pegar as tags que possuem a classe indicador
    indicadores = soup.select(".indicador")
    
    ## Em cada indicador eu tenho duas classes (label e value), então itero com um for para retirar o texto de cada um
    dict_estados = {indicador.select('.ind-label')[0].text: indicador.select('.ind-value')[0].text for indicador in indicadores}
    
    ## Criando uma coluna Estados no meu dict_estados e atribuindo a ele o valor estado
    dict_estados["Estados"] = estado
    
    ## Retornando nosso dict_estados
    return dict_estados

In [ ]:
## Criando uma lista com a UF de cada estado brasileiro p/ consultarmos suas informações

estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

In [ ]:
## Criando uma variável que recebe uma lista onde chamamos nossa função e iteramos os parâmetros com nossa lista estados

infos_estados = [consulta_info_estado(estado) for estado in estados]  

In [ ]:
## Transofmrando nossos dados em data frame do tipo pandas para tornar mais fácil sua manipulação

df = pd.DataFrame(data = infos_estados)

In [ ]:
## Verificando o tipo de dados do nosso df

type(df)

In [ ]:
## Exibindo os cinco primeiros itens do nosso data frame

df.head()

In [ ]:
## Criado nosso df temos que seus dados são do tipo objeto, logo precisamos transformá-los em números ou strings;

df.info()

#__LIMPEZA__

In [ ]:
## É uma boa prática criarmos uma cópia do nosso data frame original p/ diminuir a chance de perdermos os dados originais

df_estados = df.copy()

In [ ]:
## Renomeando as colunas do data frame adotando as boas práticas (snake_case) e separando as colunas com tipos string e dado

df_estados.columns = ['governador', 'capital', 'gentilico', 'area', 'populacao', 'densidade_demografica', 'matriculas_fundamental', 'idh', 'receitas_realizadas', 'despesas_empenhadas', 'rendimento_mensal_per_capita', 'total_veiculos', 'estados']

In [ ]:
## Aplicando um regex com o método replace p/ remover os caractéres que não queremos que sejam exibidos
## Caracteres especiais como . precisam da \ antes p/ que a regex funcione;

df_estados = df_estados.replace({
    '\.': '',
    ',': '.',
    '\[\d+\]': '',
    ' hab/km²': '',
    ' km²': '',
    ' pessoas': '',
    ' matrículas': '',
    'R\$.*': '',
    ' veículos': ''
}, regex = True)

In [ ]:
## Verificando se os caracteres foram retirados

df_estados.head()

In [ ]:
## Vou criar uma lista que contenham dados do tipo númerico para depois realizar a conversão destes

colunas_numericas = ['populacao', 'area', 'idh', 'rendimento_mensal_per_capita', 'total_veiculos', 'matriculas_fundamental', 'despesas_empenhadas', 'receitas_realizadas']

In [ ]:
## Retirando todos os possíveis espaços nas colunas_numericas com o método .apply(), lambda, .str e .strip()

df_estados[colunas_numericas] = df_estados[colunas_numericas].apply(lambda x: x.str.strip())

In [ ]:
## Convertendo as colunas_numericas de tipo string para numerico

df_estados[colunas_numericas] = df_estados[colunas_numericas].apply(pd.to_numeric)

In [ ]:
## Confirmando que as colunas tiveram seus tipos alterados para números

df_estados.info()

#__EXPORTAÇÃO__

In [ ]:
df_estados.to_csv('dados-IBGE.csv', index=False)